In [1]:
import os
import numpy as np
import nltk
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from gensim.models.word2vec import Word2Vec

Using TensorFlow backend.
/Users/zhaoxiaoxiang/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/zhaoxiaoxiang/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/zhaoxiaoxiang/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/zhaoxiaoxiang/anaconda3/l

In [2]:
raw_text = ''

In [5]:
for file in os.listdir("/Users/zhaoxiaoxiang/kaggle/七月在线/class6/代码及资料"):
    if file.endswith(".txt"):
        raw_text += open("/Users/zhaoxiaoxiang/kaggle/七月在线/class6/代码及资料/"+file, errors='ignore').read() + '\n\n'

In [7]:
raw_text = raw_text.lower()

In [9]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/zhaoxiaoxiang/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
sentensor = nltk.data.load('tokenizers/punkt/english.pickle') 

In [12]:
sents = sentensor.tokenize(raw_text)

In [13]:
sents

['\ufeffproject gutenberg’s real soldiers of fortune, by richard harding davis\n\nthis ebook is for the use of anyone anywhere at no cost and with\nalmost no restrictions whatsoever.',
 'you may copy it, give it away or\nre-use it under the terms of the project gutenberg license included\nwith this ebook or online at www.gutenberg.org\n\n\ntitle: real soldiers of fortune\n\nauthor: richard harding davis\n\nposting date: february 22, 2009 [ebook #3029]\nlast updated: september 26, 2016\n\nlanguage: english\n\ncharacter set encoding: utf-8\n\n*** start of this project gutenberg ebook real soldiers of fortune ***\n\n\n\n\nproduced by david reed, and ronald j. wilson\n\n\n\n\n\nreal soldiers of fortune\n\n\nby richard harding davis\n\n\n\n\n\nmajor-general henry ronald douglas maciver\n\nany sunny afternoon, on fifth avenue, or at night in the _table d’hote_\nrestaurants of university place, you may meet the soldier of fortune who\nof all his brothers in arms now living is the most remarka

In [14]:
corpus = []
for sen in sents:
    corpus.append(nltk.word_tokenize(sen))

print(len(corpus))
print(corpus[:3])

1792
[['\ufeffproject', 'gutenberg', '’', 's', 'real', 'soldiers', 'of', 'fortune', ',', 'by', 'richard', 'harding', 'davis', 'this', 'ebook', 'is', 'for', 'the', 'use', 'of', 'anyone', 'anywhere', 'at', 'no', 'cost', 'and', 'with', 'almost', 'no', 'restrictions', 'whatsoever', '.'], ['you', 'may', 'copy', 'it', ',', 'give', 'it', 'away', 'or', 're-use', 'it', 'under', 'the', 'terms', 'of', 'the', 'project', 'gutenberg', 'license', 'included', 'with', 'this', 'ebook', 'or', 'online', 'at', 'www.gutenberg.org', 'title', ':', 'real', 'soldiers', 'of', 'fortune', 'author', ':', 'richard', 'harding', 'davis', 'posting', 'date', ':', 'february', '22', ',', '2009', '[', 'ebook', '#', '3029', ']', 'last', 'updated', ':', 'september', '26', ',', '2016', 'language', ':', 'english', 'character', 'set', 'encoding', ':', 'utf-8', '***', 'start', 'of', 'this', 'project', 'gutenberg', 'ebook', 'real', 'soldiers', 'of', 'fortune', '***', 'produced', 'by', 'david', 'reed', ',', 'and', 'ronald', 'j.', 

In [17]:
w2v_model = Word2Vec(corpus, size=128, window=5, min_count=5, workers=4)

In [18]:
w2v_model

In [19]:
raw_input = [item for sublist in corpus for item in sublist]
len(raw_input)

55562

In [21]:
raw_input[12]

'davis'

In [26]:
text_stream = []
vocab = w2v_model.wv.vocab
for word in raw_input:
    if word in vocab:
        text_stream.append(word)
len(text_stream)

46546

In [27]:
seq_length = 10
x = []
y = []
for i in range(0, len(text_stream) - seq_length):

    given = text_stream[i:i + seq_length]
    predict = text_stream[i + seq_length]
    x.append(np.array([w2v_model[word] for word in given]))
    y.append(w2v_model[predict])

/Users/zhaoxiaoxiang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/Users/zhaoxiaoxiang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


In [29]:
print(len(x))
print(len(y))
print(len(x[12]))
print(len(x[12][0]))
print(len(y[12]))

46536
46536
10
128
128


In [30]:
x = np.reshape(x, (-1, seq_length, 128))
y = np.reshape(y, (-1,128))

In [31]:
model = Sequential()
model.add(LSTM(256, dropout_W=0.2, dropout_U=0.2, input_shape=(seq_length, 128)))
model.add(Dropout(0.2))
model.add(Dense(128, activation='sigmoid'))
model.compile(loss='mse', optimizer='adam')

Instructions for updating:
Colocations handled automatically by placer.


/Users/zhaoxiaoxiang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(256, input_shape=(10, 128), dropout=0.2, recurrent_dropout=0.2)`
  


In [32]:
model.fit(x, y, nb_epoch=50, batch_size=4096)

Instructions for updating:
Use tf.cast instead.


/Users/zhaoxiaoxiang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/50
46536/46536 [==============================] - 17s 361us/step - loss: 0.2101
Epoch 2/50
46536/46536 [==============================] - 15s 331us/step - loss: 0.0654
Epoch 3/50
46536/46536 [==============================] - 15s 328us/step - loss: 0.0592
Epoch 4/50
46536/46536 [==============================] - 16s 336us/step - loss: 0.0582
Epoch 5/50
46536/46536 [==============================] - 16s 348us/step - loss: 0.0578
Epoch 6/50
46536/46536 [==============================] - 45s 964us/step - loss: 0.0576
Epoch 7/50
46536/46536 [==============================] - 72s 2ms/step - loss: 0.0574
Epoch 8/50
46536/46536 [==============================] - 18s 386us/step - loss: 0.0572
Epoch 9/50
46536/46536 [==============================] - 18s 379us/step - loss: 0.0571
Epoch 10/50
46536/46536 [==============================] - 17s 361us/step - loss: 0.0569
Epoch 11/50
46536/46536 [==============================] - 16s 346us/step - loss: 0.0568
Epoch 12/50
46536/46536 [=======

In [33]:
def predict_next(input_array):
    x = np.reshape(input_array, (-1,seq_length,128))
    y = model.predict(x)
    return y

def string_to_index(raw_input):
    raw_input = raw_input.lower()
    input_stream = nltk.word_tokenize(raw_input)
    res = []
    for word in input_stream[(len(input_stream)-seq_length):]:
        res.append(w2v_model[word])
    return res

def y_to_word(y):
    word = w2v_model.most_similar(positive=y, topn=1)
    return word

In [34]:
def generate_article(init, rounds=30):
    in_string = init.lower()
    for i in range(rounds):
        n = y_to_word(predict_next(string_to_index(in_string)))
        in_string += ' ' + n[0][0]
    return in_string

In [39]:
init = 'Language Models allow us to measure how likely a word is, which is an important for machine'
article = generate_article(init)
print(article)

/Users/zhaoxiaoxiang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


KeyError: "word 'machine' not in vocabulary"